### Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa.display
import librosa

from apbandpass import apbandpass

### Frequency Response

In [ ]:
sr = 1000000
x = np.zeros(2*sr)
x[0]=1
fc = 500  # center freq
fb = 10000 # band width
y = apbandpass(x, 2*fc/sr, 2*fb/sr)
fft_y = np.fft.rfft(y)
fft_y_dB = librosa.amplitude_to_db(np.abs(fft_y), ref=np.max)
f = np.linspace(0, fft_y_dB.size, fft_y_dB.size)

limiter = sr/10

min_index = int(limiter)
fft_min = fft_y_dB[min_index]
if int(fft_y_dB[1]) < fft_y_dB[min_index]: fft_min = int(fft_y_dB[1])

fig, ax = plt.subplots(figsize=(16,8))
ax.semilogx(f, fft_y_dB)
plt.title(r"Impulse Response")
plt.ylabel(r"gain (dB) $\rightarrow$")
plt.xlabel(r"f $\rightarrow$")
plt.xlim(1, limiter)
plt.ylim(fft_min,10)
ax.grid()

### Test

In [ ]:
x, sr = librosa.load("./audios/deck_the_halls.wav", 44100)

x_max = x.max()
x = x/x_max

D_x = librosa.stft(x)  # STFT of x
S_db_x = librosa.amplitude_to_db(np.abs(D_x), ref=np.max)

fig, ax = plt.subplots(figsize=(16,8))
img = librosa.display.specshow(S_db_x, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Unfiltered Spectrogram')
fig.colorbar(img, ax=ax, format="%+2.f dB")

fc = 15000
fb = 200
y = apbandpass(x, 2*fc/sr, 2*fb/sr)

D_y = librosa.stft(y)  # STFT of y
S_db_y = librosa.amplitude_to_db(np.abs(D_y), ref=np.max)

fig, ax = plt.subplots(figsize=(16,8))
img = librosa.display.specshow(S_db_y, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Filtered Spectrogram')
fig.colorbar(img, ax=ax, format="%+2.f dB")

sf.write("./audios/deck_the_halls_rewrite_AP_BAND.wav", y*x_max, 44100, 'PCM_24')